In [ ]:
%matplotlib inline

In [ ]:
!pip install netCDF4
!wget "https://gtvault-my.sharepoint.com/:u:/g/personal/amagalhaes7_gatech_edu/EdUEdGuPtGRBpnFaZPApf40B4ZVdrup2o5JXkg-UKpIe7Q?e=dbHDm2&download=1" -O "CMEMS_horizontal_current_velocity_data.nc"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
from netCDF4 import Dataset
from IPython.display import HTML, display
rc('animation', html='jshtml')
import sys

In [ ]:
# Simulation parameters

# Total simulation time (days)
days_of_simulation = 300

# Day of the year to query velocity data (number of days since January 1)
day_of_year = 0

# Animation frame rate
frame_rate = 30

In [ ]:
# Model parameters

# dt, dx, and dy
dt = 3600  # Time step (seconds)
dx, dy = 9600, 9600  # Spatial step sizes (m). Each grid cell in the simulation represents a 9.6 km^2 patch of ocean

# Stokes-Einstein parameters
temp = 298.15  # Ocean temperature (K)
viscosity = 0.001  # Viscosity of ocean water (Pa·s)
radius = 1e-9  # Microplastic particle radius (m)

# Microplastic sources
source_cells = [(205, 152)]  # This source is at the mouth of the Mississippi river: 29.1511 N, -89.2533 W.

In [ ]:
def update_concentration_parallel(ocean_map, land_mask, S, Dx, Dy, u, v, dt, dx, dy, iterations):
    rows, cols = ocean_map.shape
    maps_over_time = [np.copy(ocean_map)]
    for iteration in range(iterations):
        C_current = ocean_map

        # Initialize C_new using the updated formula
        u_shift_x = np.where(u > 0, 1, -1)
        v_shift_y = np.where(v > 0, 1, -1)

        # Calculate the mask for valid neighbors
        valid_u_mask = np.where(np.roll(land_mask, -1, axis=0) == False, 1, 0)
        valid_v_mask = np.where(np.roll(land_mask, -1, axis=1) == False, 1, 0)

        # Calculate C_new with the updated rule
        C_new = (
            (1 - 2 * dt * Dx / dx**2 - 2 * dt * Dy / dy**2)
            - (np.abs(u) * valid_u_mask * dt / dx)
            - (np.abs(v) * valid_v_mask * dt / dy)
        ) * C_current

        # Apply the pollution source term during the first half of iterations
        pollution_source = np.where(iteration < iterations / 2, S * dt, 0)
        C_new += pollution_source

        # Calculate contributions from neighboring cells (same logic as before)
        mask_x_positive = (np.roll(land_mask, -1, axis=0) == False) & ((np.arange(rows) + 1 < rows)[:, np.newaxis])
        u_x_positive = np.roll(u, -1, axis=0)  # u[i+1] for x+ direction
        C_new = C_new + (dt / dx) * (Dx / dx - np.where(u_x_positive < 0, u_x_positive, 0)) * np.roll(ocean_map, -1, axis=0) * mask_x_positive

        mask_x_negative = (np.roll(land_mask, 1, axis=0) == False) & ((np.arange(rows) - 1 >= 0)[:, np.newaxis])
        u_x_negative = np.roll(u, 1, axis=0)  # u[i-1] for x- direction
        C_new = C_new + (dt / dx) * (Dx / dx + np.where(u_x_negative > 0, u_x_negative, 0)) * np.roll(ocean_map, 1, axis=0) * mask_x_negative

        mask_y_positive = (np.roll(land_mask, -1, axis=1) == False) & ((np.arange(cols) + 1 < cols)[np.newaxis, :])
        v_y_positive = np.roll(v, -1, axis=1)  # v[j+1] for y+ direction
        C_new = C_new + (dt / dy) * (Dy / dy - np.where(v_y_positive < 0, v_y_positive, 0)) * np.roll(ocean_map, -1, axis=1) * mask_y_positive

        mask_y_negative = (np.roll(land_mask, 1, axis=1) == False) & ((np.arange(cols) - 1 >= 0)[np.newaxis, :])
        v_y_negative = np.roll(v, 1, axis=1)  # v[j-1] for y- direction
        C_new = C_new + (dt / dy) * (Dy / dy + np.where(v_y_negative > 0, v_y_negative, 0)) * np.roll(ocean_map, 1, axis=1) * mask_y_negative

        # Update the ocean_map
        ocean_map = np.where(land_mask, -np.max(C_new), C_new)

        # Append 1 iteration per day to maps_over_time
        if iteration % (86400 / dt) == 0:
            maps_over_time.append(np.copy(ocean_map))

        # if iteration % 10 == 0:
        #     maps_over_time.append(np.copy(ocean_map))

    return maps_over_time


def animate_concentration(maps_over_time, frame_rate):
    fig, ax = plt.subplots(figsize=(6, 6))
    cmap = 'viridis'
    cax = ax.imshow(maps_over_time[0], cmap=cmap, origin='lower')
    fig.colorbar(cax, ax=ax, label='Concentration')

    def update(frame):
        cax.set_data(maps_over_time[frame])
        ax.set_title(f'Concentration Map - Day {frame}')

    anim = animation.FuncAnimation(fig, update, frames=len(maps_over_time), interval=1000 / frame_rate)

    plt.close()

    return anim


def stokes_einstein(T, eta, r):
    """
    Calculate the diffusion coefficient using the Stokes-Einstein equation.

    Parameters:
        T: float, demperature in Kelvin (K)
        eta: float, dynamic viscosity of the fluid (N·s/m²)
        r: float, radius of the particle in meters (m)

    Returns:
    Float, diffusion coefficient (m²/s)
    """
    k_B = 1.380649e-23  # Boltzmann constant in J/K
    D = k_B * T / (6 * np.pi * eta * r)

    return D

In [ ]:
%%time

# Load current velocity data in order to determine u and v parameters of PDE
velocity_dataset = Dataset("./CMEMS_horizontal_current_velocity_data.nc")
uo_data = np.array(velocity_dataset.variables['uo'][:])  # Shape: (time, depth, latitude, longitude)
vo_data = np.array(velocity_dataset.variables['vo'][:])  # Shape: (time, depth, latitude, longitude)
velocity_dataset.close()

# Preprocess data
uo_data = np.squeeze(uo_data)
vo_data = np.squeeze(vo_data)
uo_data = uo_data[:, :240, :]  # Trim off one excess layer
vo_data = vo_data[:, :240, :]  # Trim off one excess layer

# Use data for u and v arrays
u, v = uo_data[day_of_year], vo_data[day_of_year]  # (m/s)

# Determine map size
rows, cols = u.shape[0], u.shape[1]

# Calculate the number of iterations needed given simulation time (days) and time step (seconds) values
iterations = int((days_of_simulation * 86400) / dt)

# Calculate diffusion parameters Dx, Dy of PDE
Dx = Dy = 10000 * stokes_einstein(temp, viscosity, radius) * dx * dy  # Assuming isotropic. Diffusion also scales with dx * dy

# Generate initial ocean map
ocean_map = np.zeros((rows, cols))

# Generate dirichlet boundary conditions using velocity data (NaNs in data represent land)
land_mask = np.zeros((rows, cols), dtype=bool)
missing_uo_indices = np.isnan(uo_data[0])
land_mask[missing_uo_indices] = True

u[np.isnan(u)] = 0
v[np.isnan(v)] = 0

# Add sources
S = np.zeros((rows, cols))
for source in source_cells:
    S[source[0], source[1]] = 100

In [ ]:
# Update concentration and collect maps over time
maps_over_time = update_concentration_parallel(ocean_map, land_mask, S, Dx, Dy, u, v, dt, dx, dy, iterations)

# Animate the concentration map
anim = animate_concentration(maps_over_time, frame_rate=5)
display(HTML(anim.to_jshtml()))

In [ ]:
plt.figure(figsize=(10, 6))

plt.imshow(land_mask, cmap="Greys", interpolation="nearest")
plt.colorbar(label="Land/Sea")
plt.gca().invert_yaxis()

plt.scatter(153, 206, color="red", s=10)

plt.show()